In [1]:
import tensorflow as tf
import numpy as np
import os
import time
%config Completer.use_jedi = False


from tensorflow import keras
from tensorflow.keras import layers

In [2]:
text = np.load("fish_mur2.npy")
# The unique characters in the file
vocabulary = sorted(set(text))
print ('{} unique characters.'.format(len(vocabulary)))

vocabulary_size = len(vocabulary)

c2emb = {char:index for index, char in enumerate(vocabulary)}
emb2c = np.array(vocabulary)

1313 unique characters.


In [3]:
sequence_length = 600
examples_per_epoch = len(text)//sequence_length
text_emb = np.array([c2emb[code] for code in text])
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_emb)

In [4]:
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

for item in sequences.take(2):
      print(item.numpy())

[ 642  642  639  634  642  642  634  634  634  634  642  642  988  987
 1212 1212 1213 1211 1213 1213 1213  985  985  985  985  985  985  642
  642  642  642  642  647  642  642  642  642  642  647  642  988  984
 1212 1213 1211 1211 1213 1213  985 1213 1213  985  985  985  985  985
  985  985  985  985  985  642  642  642  642  642  642  642  642  642
  642  642  962  957  954 1291 1193 1189  959  956  956  956  959  959
  959  959  959  959  959  979  639  592  588  588  956  956  634  634
  634  639  634  588  588  136  144  586  586  586  586  142  153  132
  148  592  639  981 1209  987  984  641  641  700  700 1234 1234 1014
 1015 1011  701  701 1011 1235 1011 1011 1011  701 1011  701  701  701
  701  701  701  701  650  648  993  891 1109 1225 1003 1011 1011  701
  695  695  701  695  695  701  695  695  701  695  695  695  695  695
  260  695  642  986  986  983 1213 1213  984  985  985  985  642  642
  642  642  642  642  695  642  695  642  202  260  847 1112  849  745
  851 

In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [6]:
# Batch size
batch = 64
steps_per_epoch = examples_per_epoch//batch

# TF data maintains a buffer in memory in which to shuffle data
# since it is designed to work with possibly endless data
buffer = 10000

dataset = dataset.shuffle(buffer).batch(batch, drop_remainder=True)

dataset = dataset.repeat()

dataset

<RepeatDataset shapes: ((64, 600), (64, 600)), types: (tf.int64, tf.int64)>

In [7]:
#  The vocabulary length in characterrs
vocabulary_length = len(vocabulary)

# The embedding dimension
embedding_dimension = 512

# Number of RNN units
recurrent_nn_units = 1024

In [8]:
if tf.test.is_gpu_available():
    recurrent_nn = tf.compat.v1.keras.layers.CuDNNGRU
    print("GPU in use")
else:
    import functools
    recurrent_nn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
    print("CPU in use")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU in use


In [9]:
def build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size):
    model = tf.keras.Sequential(
        [tf.keras.layers.Embedding(vocabulary_size, embedding_dimension, batch_input_shape=[batch_size, None]),
    recurrent_nn(recurrent_nn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
    tf.keras.layers.Dense(vocabulary_length)
  ])
    return model

In [11]:
model = build_model(
  vocabulary_size = len(vocabulary),
  embedding_dimension=embedding_dimension,
  recurrent_nn_units=recurrent_nn_units,
  batch_size=batch)

In [14]:
for batch_input_example, batch_target_example in dataset.take(1):
    batch_predictions_example = model(batch_input_example)
    print(batch_predictions_example.shape, "# (batch, sequence_length, vocabulary_length)")

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           672256    
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          4724736   
_________________________________________________________________
dense_1 (Dense)              (64, None, 1313)          1345825   
Total params: 6,742,817
Trainable params: 6,742,817
Non-trainable params: 0
_________________________________________________________________


In [174]:
sampled_indices = tf.random.categorical(logits=batch_predictions_example[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [175]:
sampled_indices

array([1037, 1255, 1039, 1016,  328,  933,  280,  183,  329,  336,  469,
       1094,  873,  819,  877, 1287,  469,  878,  874,  874,  877,  823,
        469, 1238,  819, 1088,  406,  821,  821,  821,  821,  821,  469,
        469, 1287,  874,  521, 1124,  406,  821,  821, 1307, 1124,  867,
       1305,  461,  324,  821,  867,  406, 1305,  867,  867,  461, 1286,
        956, 1275, 1079,  810, 1274, 1278, 1275,  918,  762,  810,    8,
        741,  762,  837,  860,  580, 1152,  268,  753,  827,   36,  412,
       1250, 1309, 1028,  743,  751, 1238,  744,  709, 1232, 1235, 1018,
        752,  707,  487, 1177, 1015,  525,  709,  697,  772, 1037,  753,
        753,  751,  744,  855,  751,  744, 1232, 1036, 1037,  760,  173,
        852,  744, 1033, 1216, 1017, 1253,  705, 1011,  267, 1009])

In [176]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [177]:
batch_loss_example  = tf.compat.v1.losses.sparse_softmax_cross_entropy(batch_target_example, batch_predictions_example)
print("Prediction shape: ", batch_predictions_example.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", batch_loss_example.numpy())

Prediction shape:  (64, 120, 1313)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.2654853


In [178]:
#next produced by upgrade script.... 
#model.compile(optimizer = tf.compat.v1.train.AdamOptimizer(), loss = loss) 
#.... but following optimizer is available.
model.compile(optimizer = tf.optimizers.Adam(), loss = loss)

In [179]:
# Directory where the checkpoints will be saved
directory = './checkpoints-copy'
# Name of the checkpoint files
file_prefix = os.path.join(directory, "ckpt_{epoch}")

callback=[tf.keras.callbacks.ModelCheckpoint(filepath=file_prefix, save_weights_only=True)]


In [180]:
epochs=500

In [181]:
history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=callback)

Epoch 1/500


ValueError: Expect x to be a non-empty array or dataset.

In [25]:
tf.train.latest_checkpoint(directory)

'./checkpoints-copy/ckpt_300'

In [42]:
model = build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(directory))

model.build(tf.TensorShape([1, None]))

In [27]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 512)            672256    
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (1, None, 1024)           4724736   
_________________________________________________________________
dense_2 (Dense)              (1, None, 1313)           1345825   
Total params: 6,742,817
Trainable params: 6,742,817
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string, temperature, characters_to_generate):

  # Vectorise  start string into numbers
    input_string = [c2emb[code] for code in start_string]
    input_string = tf.expand_dims(input_string, 0)

  # Empty string to store  generated text
    generated = start_string
    
  # (Batch size is 1)
    model.reset_states()
    for i in range(characters_to_generate):
        predictions = model(input_string)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(logits=predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted word as the next input to the model
      # along with the previous hidden state
        input_string = tf.expand_dims([predicted_id], 0)

        generated.append(emb2c[predicted_id])

    return generated # generated is a list

In [39]:
generated_text = generate_text(model=model, start_string=[text[np.random.randint(len(text))]], temperature=0.1, characters_to_generate = 12000)
#print(generated_text)

In [40]:
np.save("fish_murme2.npy", generated_text)

In [41]:
np.random.randint(len(text))

21486